In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score


In [27]:
# 데이터 불러오기
df = pd.read_csv("D:/workspace/project/data/filtered_books_utf8.csv", encoding="utf-8")

C:\Users\user\AppData\Local\Temp\ipykernel_28600\3956447301.py:2: DtypeWarning: Columns (4,5,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("D:/workspace/project/data/filtered_books_utf8.csv", encoding="utf-8")


In [ ]:
kdc_to_major = {
    # 인문계열
    "국어국문학과": ["710", "810"],
    "영어영문학과": ["740", "820"],
    "중어중문학과": ["720", "820"],
    "독어독문학과": ["730", "830"],
    "불어불문학과": ["750", "840"],
    "사학과": ["900"],
    "철학과": ["100"],
    "문예창작학과": ["810", "820"],
    "언어학과": ["700"],

    # 사회계열
    "정치외교학과": ["340"],
    "행정학과": ["350"],
    "사회학과": ["360"],
    "심리학과": ["180"],
    "언론정보학과": ["320", "070"],
    "경제학과": ["320"],
    "경영학과": ["325"],
    "회계학과": ["325.9"],
    "국제통상학과": ["321.9"],

    # 교육계열
    "교육학과": ["370"],
    "국어교육과": ["373.4"],
    "영어교육과": ["373.45"],
    "수학교육과": ["373.53"],
    "역사교육과": ["373.9"],
    "초등교육과": ["372"],
    "특수교육과": ["378"],

    # 공학계열
    "기계공학과": ["530"],
    "전기전자공학과": ["539"],
    "컴퓨터공학과": ["004", "005"],
    "화학공학과": ["542"],
    "건축공학과": ["540"],
    "토목환경공학과": ["528", "543"],
    "산업공학과": ["528.9"],
    "정보통신공학과": ["547"],
    "재료공학과": ["535"],

    # 자연계열
    "수학과": ["410"],
    "통계학과": ["310.76", "419"],
    "물리학과": ["420"],
    "화학과": ["430"],
    "생명과학과": ["470"],
    "지구과학과": ["450"],

    # 의약계열
    "의학과": ["510"],
    "간호학과": ["513"],
    "치의학과": ["517"],
    "한의학과": ["519"],
    "약학과": ["517.5"],
    "수의학과": ["518"],
    "보건행정학과": ["514.1"],

    # 예체능계열
    "음악학과": ["670"],
    "미술학과": ["650"],
    "디자인학과": ["653"],
    "연극영화학과": ["688"],
    "영화영상학과": ["688.2"],
    "스포츠과학과": ["692"],
    "체육교육학과": ["692.07"],
    "무용학과": ["689"]
}

In [28]:
# 주어진 학과-분류번호 매핑 딕셔너리를 문자열로 구성한 상태라고 가정
kdc_to_major_inv = {}
for major, codes in kdc_to_major.items():
    for code in codes:
        code_str = str(code).zfill(3)  # '004'처럼 보존
        kdc_to_major_inv.setdefault(code_str, []).append(major)

In [29]:
# 분류번호 앞 공백 제거 및 '004'처럼 표현
df["주제분류번호"] = df["주제분류번호"].astype(str).str.strip().str.zfill(3)

# 학과 리스트 추론
df["관련학과들"] = df["주제분류번호"].map(kdc_to_major_inv)

# 관련 학과가 없는 경우 제외
df = df[df["관련학과들"].map(lambda x: isinstance(x, list) and len(x) > 0)]

In [30]:
# 입력: 도서명 + 주제분류번호를 하나의 텍스트로 합침
df["입력문장"] = df["도서명"].str.strip() + " [KDC:" + df["주제분류번호"] + "]"

In [31]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df["관련학과들"])

In [32]:
from sklearn.model_selection import train_test_split

X = df["입력문장"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [33]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

pipeline = Pipeline([
    ("vect", TfidfVectorizer(ngram_range=(1, 2))),
    ("clf", OneVsRestClassifier(LogisticRegression(max_iter=1000)))
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect', TfidfVectorizer(ngram_range=(1, 2))),
                ('clf',
                 OneVsRestClassifier(estimator=LogisticRegression(max_iter=1000)))])

In [34]:
def 예측학과(도서명, 분류번호):
    query = 도서명.strip() + " [KDC:" + str(분류번호).zfill(3) + "]"
    pred = pipeline.predict([query])
    return mlb.inverse_transform(pred)[0]

In [46]:
예측학과("AI 시대의 회계정보시스템", "325")  


('회계학과',)

In [ ]:
예측학과("디자인 감각", "658")

()